In [150]:
from pathlib import Path

import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import numpy as np
from sklearn.model_selection import train_test_split

import pyro
from pyro.distributions import Categorical, Bernoulli
from pyro import param
from pyro.infer import SVI, Trace_ELBO
from pyro.optim import Adam
from pyro.infer import MCMC, NUTS
from pyro.distributions import constraints
import pyro.distributions as dist
from pyro.infer import Predictive

In [32]:
feature_path = Path.cwd().parent / 'train_test_features.parquet'
data = pd.read_parquet(feature_path) 
train, test = train_test_split(data, test_size=0.2) 

In [86]:
def discretize(data):
    '''
    transforms continuous data in the range [0,1] and
    no data values of  -99 to integers from 0 to 11.
    Where 11 is no data value'''
    # Handle the special case
    special_case = (data == -99).int() * 11
    discretized = (torch.clamp(torch.round(data * 10).int(), 0, 10) + special_case).long()
    return discretized

ahh

In [87]:
# features, found in mortality_classification.ipynb
# based on abs(corr) < 0.4, and RF feature importance
feature_names = [
    'n_mean',
    'lum70',
    'savi_std',
    'rgi_std',
    'lum50',
    'lum30',
    'b_std',
    'r_std',
    'r40',
    'lum40',
    'g30',
    'b50',
    'lum10',
    'b10',
    'n10',
    'rgi60',
    'rgi80',
    'r60',
    'b80',
    'rgi30',
    'r70',
    'n80',
    'b60'
 ]

features = torch.tensor(
    train[feature_names].values,
    dtype=torch.float32
)

labels = torch.tensor(
    train.y.values,
    dtype=torch.long
    )


23

In [153]:
# define model
def model(features, p, labels=None):
    n_samples, n_features = features.shape
    
    # If p_prior is a scalar, we replicate it to match the batch shape.
    probs = p * torch.ones(n_samples, dtype=torch.float32)
    
    with pyro.plate('data', size=n_samples):
        # capture samples of p for the posterior
        pyro.sample('p_aux', dist.Delta(probs), obs=probs)
        
        if labels is not None:
            y = pyro.sample('y', Bernoulli(probs), obs=labels.float())
        else:
            y = pyro.sample('y', Bernoulli(probs))
        
        for i in range(n_features):
            discretized_feature = discretize(features[:, i])
            pyro.sample(f'obs_{i}', Categorical(probs=torch.ones(12)/12), obs=discretized_feature)


# make a nn guide class
class NNGuide(nn.Module):
    def __init__(self, input_size, hidden_size=10):
        super(NNGuide, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc_mu = nn.Linear(hidden_size, 1)  # Outputs the mean of the normal distribution
        self.fc_sigma = nn.Linear(hidden_size, 1)  # Outputs the log standard deviation

    def forward(self, features):
        x = torch.relu(self.fc1(features))
        mu = self.fc_mu(x)
        sigma = F.softplus(self.fc_sigma(x))  # Ensure sigma is positive
        return mu, sigma


# instantiate nn guide
guide_model = NNGuide(features.shape[-1])


# def model guide
def guide(features, p, labels=None):
    mu, sigma = guide_model(features)
    mu = mu.squeeze(-1)
    sigma = sigma.squeeze(-1)
    
    with pyro.plate('data', size=features.shape[0]):
        pyro.sample('p_aux', dist.LogNormal(mu, sigma))
    

# train model and guide using SVI
optimizer = Adam({'lr': 0.01})
svi = SVI(model, guide, optimizer, loss=Trace_ELBO())


# prior based on labels
p_prior = float(train.y.sum() / len(train.y))

labels_reshaped = labels.unsqueeze(-1)

num_epochs = 5000 

for epoch in range(num_epochs):
    loss = svi.step(features.float(), p_prior, labels.long())
    if epoch % 100 == 0:  # print loss every 100 steps
        print(f'Epoch {epoch} Loss: {loss}')



/home/michael/TreeMortality/venvs/pyro/lib/python3.10/site-packages/pyro/util.py:288: UserWarning: Found non-auxiliary vars in guide but not model, consider marking these infer={'is_auxiliary': True}:
{'p_aux'}
  warnings.warn(


Epoch 0 Loss: 573973.5424804688
Epoch 100 Loss: 573830.1557617188
Epoch 200 Loss: 573947.1508789062
Epoch 300 Loss: 573984.6938476562
Epoch 400 Loss: 573915.4985351562
Epoch 500 Loss: 574005.5385742188
Epoch 600 Loss: 573904.9965820312
Epoch 700 Loss: 573887.6791992188
Epoch 800 Loss: 573870.2856445312
Epoch 900 Loss: 573946.1811523438
Epoch 1000 Loss: 573909.9057617188
Epoch 1100 Loss: 573804.8002929688
Epoch 1200 Loss: 573880.7661132812
Epoch 1300 Loss: 574051.5874023438
Epoch 1400 Loss: 574029.0844726562
Epoch 1500 Loss: 574011.7563476562
Epoch 1600 Loss: 573993.4711914062
Epoch 1700 Loss: 573923.0805664062
Epoch 1800 Loss: 573939.5063476562
Epoch 1900 Loss: 573997.3471679688
Epoch 2000 Loss: 573852.9780273438
Epoch 2100 Loss: 573845.9204101562
Epoch 2200 Loss: 573982.3559570312
Epoch 2300 Loss: 573846.6596679688
Epoch 2400 Loss: 574211.8256835938
Epoch 2500 Loss: 573914.2729492188
Epoch 2600 Loss: 573828.7202148438
Epoch 2700 Loss: 573957.3637695312
Epoch 2800 Loss: 574006.08056640

In [154]:
test_features = torch.tensor(
    test[feature_names].values,
    dtype=torch.float32
)

test_labels = torch.tensor(
    test.y.values,
    dtype=torch.long
    )

In [156]:
# pass the new data through the trained guide model
with torch.no_grad():
    post_mu, post_sigma = guide_model(test_features)

In [157]:
post_mu

tensor([[-0.0824],
        [13.0301],
        [-0.0310],
        ...,
        [-0.0480],
        [-0.0272],
        [-0.0365]])

In [143]:
num_samples = 1000

predictive = Predictive(model, guide=guide, num_samples=num_samples, return_sites=['y', 'p_aux'])

samples = predictive(test_features, p_prior)

In [144]:
samples['p_aux']

tensor([[0.7693, 0.7693, 0.7693,  ..., 0.7693, 0.7693, 0.7693],
        [0.7693, 0.7693, 0.7693,  ..., 0.7693, 0.7693, 0.7693],
        [0.7693, 0.7693, 0.7693,  ..., 0.7693, 0.7693, 0.7693],
        ...,
        [0.7693, 0.7693, 0.7693,  ..., 0.7693, 0.7693, 0.7693],
        [0.7693, 0.7693, 0.7693,  ..., 0.7693, 0.7693, 0.7693],
        [0.7693, 0.7693, 0.7693,  ..., 0.7693, 0.7693, 0.7693]])

In [139]:
y_samp = samples['p_aux']
y_samp

KeyError: 'p_aux'

tensor(0.5310)